In [29]:
import uproot
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from tqdm.auto import tqdm
import torch.cuda.amp as amp

In [16]:
# Load datasets
# #paths to samples
zero_t_samples = [
    'data/WW.Flz.part1.root',
    'data/WW.Flz.part2.root',
    'data/Wjets-incl.INb.part1.root',
    'data/Wjets-incl.INb.part2.root',
    'data/Wjets-incl.INb.part3.root',
    'data/Wjets-incl.INb.part4.root',
    'data/Wjets-incl.INb.part5.root',
]

one_t_samples = [
    'data/t-channel_tbar_4f.leB.part1.root',
    'data/t-channel_tbar_4f.leB.part2.root',
    'data/t-channel_top_4f.qlh.part1.root',
    'data/t-channel_top_4f.qlh.part2.root',
]

two_t_samples = [
    'data/ttbar_doublelep.KQP.part1.root',
    'data/ttbar_doublelep.KQP.part2.root',
    'data/ttbar_doublelep.KQP.part3.root',
    'data/ttbar_doublelep.KQP.part4.root',
    'data/ttbar_semilep.vRt.part1.root',
    'data/ttbar_semilep.vRt.part2.root',
]

three_t_samples = [
    'data/TTTJ_UL16Summer20_RoO.root',
    'data/TTTW_UL16Summer20_CgT.root',
]

four_t_samples = [
    'data/TTTT_UL16Summer20_eIE.part1.root',
    'data/TTTT_UL16Summer20_eIE.part2.root',
    'data/TTTT_UL16Summer20_eIE.part3.root',
    'data/TTTT_UL16Summer20_eIE.part4.root',
]

# create dataframe
total_data = []
for i, samples in tqdm(enumerate([zero_t_samples,one_t_samples,two_t_samples,three_t_samples,four_t_samples])):
    list_of_samples = []
    for path in tqdm(samples):
        print('processing',i, path)
        file = uproot.open(path)
        a = file['LHE'].arrays(file['LHE'].keys(), library="np")
        df = pd.DataFrame(a)
        list_of_samples.append(df)
        print('successful')
    df_sample = pd.concat(list_of_samples).reset_index()
    df_sample['class'] = i
    total_data.append(df_sample)
data = pd.concat(total_data).reset_index()
print('shape:', data.shape)

0it [00:00, ?it/s]

processing 0 data/WW.Flz.part1.root


successful
processing 0 data/WW.Flz.part2.root


successful
processing 0 data/Wjets-incl.INb.part1.root


successful
processing 0 data/Wjets-incl.INb.part2.root


...

successful
processing 4 data/TTTT_UL16Summer20_eIE.part4.root


100%|██████████| 4/4 [00:19<00:00,  4.83s/it]

successful



5it [03:32, 42.50s/it]


shape: (8198428, 301)


In [51]:
data = data.drop(columns = ['level_0', 'index'])   

In [54]:
df = data.groupby('class', group_keys=False).apply(lambda x: x.sample(frac = 0.1, random_state=42))

/tmp/ipykernel_1546215/226860367.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = data.groupby('class', group_keys=False).apply(lambda x: x.sample(frac = 0.1, random_state=42))


In [17]:
def split_input_target(df):
    # Выбираем ВСЕ переменные с "Gen" (включая генераторные глобальные переменные)
    target_columns = [col for col in df.columns if 'Gen' in col]
    
    # Выбираем ВСЕ переменные БЕЗ "Gen" (конечные частицы + обычные глобальные переменные)
    input_columns = [col for col in df.columns if 'Gen' not in col]
    
    # Проверка на случай, если есть колонки, не связанные ни с чем
    remaining = set(df.columns) - set(target_columns + input_columns)
    if remaining:
        raise ValueError(f'Unaccounted columns: {remaining}')
    
    
    X = df[input_columns].copy()
    y = df[target_columns].copy()
    
    return X, y

In [55]:
x, y = split_input_target(df)

In [ ]:
class BasicRegression(nn.Module):
    def __init__(self, 
                 num_input_particles=20,   # Входные частицы
                 num_gen_particles=28,     # Генераторные частицы
                 input_features=6,         # Признаки на входную частицу
                 gen_features=6,           # Признаки на генераторную частицу
                 d_model=128, 
                 nhead=4):
        super().__init__()

        self.num_input_particles = num_input_particles
        self.num_gen_particles = num_gen_particles
        self.d_model = d_model

        # Преобразование входных частиц [B, 20, 6] → [B, 20, D]
        self.input_projection = nn.Linear(input_features, d_model)

        # Обучаемые запросы для генераторных частиц
        self.gen_queries = nn.Parameter(torch.randn(num_gen_particles, d_model))  # [28, D]

        # Multi-head attention: query = ген. частицы, key/value = входные
        self.attention = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=nhead,
            dropout=0.1,
            batch_first=True
        )

        # Предсказание параметров генераторных частиц
        self.gen_decoder = nn.Sequential(
            nn.Linear(d_model, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Linear(256, gen_features)
        )

        # Предсказание вероятностей наличия частиц
        self.presence_classifier = nn.Sequential(
            nn.LayerNorm(d_model)
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # 1 вероятность на каждую из 28 частиц
            nn.Sigmoid()
        )

        # Глобальный декодер на основе всего attention-выхода
        self.global_decoder = nn.Sequential(
            nn.Linear(num_gen_particles * d_model, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # или нужный размер
        )

    def forward(self, x):
        B = x.size(0)

        # Разделение на частицы и глобальные переменные (если есть)
        particles = x[:, :self.num_input_particles * 6].view(B, self.num_input_particles, 6)

        # Преобразуем входные признаки
        projected_particles = self.input_projection(particles)  # [B, 20, D]

        # Маска: True там, где частица есть (ненулевая)
        input_mask = (particles.abs().sum(dim=-1) > 0)  # [B, 20]

        # Повторяем gen_queries по батчу
        queries = self.gen_queries.unsqueeze(0).repeat(B, 1, 1)  # [B, 28, D]

        # Attention: [B, 28, D]
        attn_output, _ = self.attention(
            query=queries,
            key=projected_particles,
            value=projected_particles,
            key_padding_mask=~input_mask  # маска: True — не использовать
        )

        # Предсказания генераторных частиц
        gen_particles = self.gen_decoder(attn_output)  # [B, 28, gen_features]

        # Вероятность наличия частицы
        presence_probs = self.presence_classifier(attn_output).squeeze(-1)  # [B, 28]

        # Маскирование
        gen_particles = self.apply_mask(gen_particles, presence_probs)

        # Глобальные признаки — flatten всех ген. выходов
        global_features = attn_output.reshape(B, -1)  # [B, 28 * D]
        global_pred = self.global_decoder(global_features)  # [B, ?]

        return gen_particles, global_pred, presence_probs

    def apply_mask(self, particles, probs, threshold=0.5):
        mask = (probs > threshold).float().unsqueeze(-1)  # [B, 28, 1]
        return particles * mask

In [57]:
def loss_fn(pred_gen, pred_global, true_gen, true_global, presence_probs, presence_true):
    mask = presence_true.unsqueeze(-1)
    mse_particles = torch.mean(mask * (pred_gen - true_gen)**2)
    mse_global = torch.mean((pred_global - true_global)**2)
    bce = F.binary_cross_entropy(presence_probs, presence_true)
    
    total_loss = (
        0.7 * mse_particles +
        0.3 * mse_global +
        0.2 * bce
    )
    
    return total_loss, {
        'particles': mse_particles,
        'global': mse_global,
        'presence': bce
    }

In [83]:
class ParticleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        
        
        particle_types = {
            'GenBJet': 4,    # 4 b-джета
            'GenJet': 12,    # 12 джетов
            'GenLep': 4,     # 4 лептона
            'GenNu': 4,      # 4 нейтрино
            'GenTop': 4      # 4 топ-кварка
        }
        
        
        all_particles = []
        for p_type, p_count in particle_types.items():
            for i in range(1, p_count+1):
                
                particle_data = y[[
                    f'Pt_{p_type}{i}',
                    f'Eta_{p_type}{i}',
                    f'Phi_{p_type}{i}',
                    f'Px_{p_type}{i}',
                    f'Py_{p_type}{i}',
                    f'Pz_{p_type}{i}'
                ]].values
                all_particles.append(particle_data)
        
        
        self.y_gen = torch.tensor(np.stack(all_particles, axis=1), dtype=torch.float32)
        
       
        self.y_global = torch.tensor(y[['N_GenJ', 'N_GenBJ', 'N_GenNu', 'N_GenLep', 'N_GenTop']].values, 
                                    dtype=torch.float32)
        
        # Маска присутствия (по Pt > 0)
        self.presence_mask = (self.y_gen[:, :, 0] > 1e-5).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            'input': self.X[idx],
            'target_gen': self.y_gen[idx],
            'target_global': self.y_global[idx],
            'presence_mask': self.presence_mask[idx]
        }

In [59]:
def train_model(model, train_loader, val_loader, loss_fn, config):
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    best_val_loss = np.inf
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_particles': [],
        'train_global': [],
        'train_presence': [],
        'val_particles': [],
        'val_global': [],
        'val_presence': []
    }
    
    device = torch.device(config['device'])
    model = model.to(device)
    
    for epoch in tqdm(range(config['num_epochs']), desc='Epochs', position=0):
        # Training
        model.train()
        epoch_loss = 0.0
        particles_loss = 0.0
        global_loss = 0.0
        presence_loss = 0.0
        train_progress = tqdm(train_loader, desc=f'Train Epoch {epoch+1}', leave=False, position=1)
        for batch in train_progress:
            optimizer.zero_grad()
            
            inputs = batch['input'].to(device)
            true_gen = batch['target_gen'].to(device)
            true_global = batch['target_global'].to(device)
            presence_true = batch['presence_mask'].to(device)
            
            pred_gen, pred_global, pred_presence = model(inputs)
            loss, components = loss_fn(pred_gen, pred_global, true_gen,
                                      true_global, pred_presence, presence_true)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            particles_loss += components['particles'].item()
            global_loss += components['global'].item()
            presence_loss += components['presence'].item()
        
        # Save training metrics
        history['train_loss'].append(epoch_loss/len(train_loader))
        history['train_particles'].append(particles_loss/len(train_loader))
        history['train_global'].append(global_loss/len(train_loader))
        history['train_presence'].append(presence_loss/len(train_loader))
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_particles = 0.0
        val_global = 0.0
        val_presence = 0.0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input'].to(device)
                true_gen = batch['target_gen'].to(device)
                true_global = batch['target_global'].to(device)
                presence_true = batch['presence_mask'].to(device)
                
                pred_gen, pred_global, pred_presence = model(inputs)
                loss, components = loss_fn(pred_gen, pred_global, true_gen,
                                          true_global, pred_presence, presence_true)
                
                val_loss += loss.item()
                val_particles += components['particles'].item()
                val_global += components['global'].item()
                val_presence += components['presence'].item()
        avg_val_loss = val_loss/len(val_loader)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), config["model_save_path"])
            print(">> Best model saved.")
        # Save validation metrics
        history['val_loss'].append(val_loss/len(val_loader))
        history['val_particles'].append(val_particles/len(val_loader))
        history['val_global'].append(val_global/len(val_loader))
        history['val_presence'].append(val_presence/len(val_loader))
        
        # Print progress
        print(f'Epoch {epoch+1}/{config["num_epochs"]}')
        print(f'Train Loss: {history["train_loss"][-1]:.4f} | Val Loss: {history["val_loss"][-1]:.4f}')
    
    # Plot all losses on one figure
    plt.figure(figsize=(12, 6))
    
    # Total losses
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')
    plt.title('Total Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Component losses
    plt.subplot(1, 2, 2)
    plt.plot(history['train_particles'], label='Train Particles')
    plt.plot(history['val_particles'], label='Val Particles')
    plt.plot(history['train_global'], label='Train Global')
    plt.plot(history['val_global'], label='Val Global')
    plt.plot(history['train_presence'], label='Train Presence')
    plt.plot(history['val_presence'], label='Val Presence')
    plt.title('Component Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('br_loss_lr_e-4')
    plt.show()
    
    return model, history



In [104]:
config = {
    "num_epochs": 20,
    "batch_size": 256,
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "model_save_path": "basic_regressor.pt",
    "device": "cuda",
    "model_params": {
        "num_input_particles": 20,
        "d_model": 128,
        "nhead": 4
    }
}
model = BasicRegression()
model.to('cuda')

BasicRegression(
  (input_projection): Linear(in_features=6, out_features=128, bias=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (gen_decoder): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=256, out_features=6, bias=True)
  )
  (presence_classifier): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
    (3): Sigmoid()
  )
  (global_decoder): Sequential(
    (0): Linear(in_features=3584, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(x.drop(columns= 'class'), y, test_size=0.2, stratify=x['class'], shuffle=True, )

train_dataset = ParticleDataset(X_train, y_train)
val_dataset = ParticleDataset(X_val, y_val)


train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=6, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers=6, pin_memory=True)

In [107]:
trained_model, history = train_model(
        model=model,
        train_loader=train_loader,  
        val_loader=val_loader,      
        loss_fn=loss_fn,
        config=config
    )

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Train Epoch 1:   0%|          | 0/2563 [00:15<?, ?it/s]

/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [2,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [3,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [4,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [5,0,0] Assertion `input_val >= zero && input_val <= one` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [7,0,0], thread: [6,0,0] Assertion `inpu

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [68]:
sample = next(iter(train_loader))

In [88]:
sample['target_gen'].shape

torch.Size([256, 28, 6])

In [105]:
tryout = model(sample['input'].to('cuda'))

In [106]:
tryout[0].shape

torch.Size([256, 28, 6])